In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

#Dataset hyperparameters
NUM_TRAIN = 49000
batch_size = 64

#Normalize CIFAR-10 images using RGB channels' known means and stdvs
CIFAR10_transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

#Download and normalize train, val, test datasets
cifar10_train = dset.CIFAR10('./datasets', train=True, download=True, transform=CIFAR10_transform)
cifar10_val = dset.CIFAR10('./datasets', train=True, download=True, transform=CIFAR10_transform)
cifar10_test = dset.CIFAR10('./datasets', train=False, download=True, transform=CIFAR10_transform)

#Split train,val set, load datasets into usable form.
loader_train = DataLoader(cifar10_train, batch_size=batch_size,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
loader_val = DataLoader(cifar10_val, batch_size=batch_size,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN,50000)))
loader_test = DataLoader(cifar10_test, batch_size=batch_size)

#indicate whether or not to use a GPU
USE_GPU = False
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#set tensor datatypes
dtype = torch.float32

#how often to print training loss
print_every = 100

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [39]:
#flatten function for Conv --> FC layer connections
def flatten(x):
    N = x.shape[0]
    return x.view(N, -1)

In [40]:
#Basic ConvNet (module API)
class BasicNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel,channel_1,3,padding=1)
        self.conv2 = nn.Conv2d(channel_1,channel_2,3,padding=1)
        self.fc1 = nn.Linear(channel_2*32*32,num_classes)
    
    def forward(self,x):    
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = flatten(x)       
        scores = self.fc1(x)
        
        return scores        

In [41]:
#make sure our net is wired correctly, should output [64,10] tensor
def testBasicNet():
    x = torch.zeros((64,3,32,32),dtype=dtype)
    model = BasicNet(3,16,16,10)
    scores = model(x)
    print(scores.size())
    
testBasicNet()

torch.Size([64, 10])


In [42]:
#Check accuracy of model predictions. For use in training loop.
def checkAccuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on validation set")
    else:
        print("Checking accuracy")
    num_correct = 0
    num_samples = 0
    model.eval() #set model to eval mode
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device, dtype=dtype)  # move to GPU if in use
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _,preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        accuracy = float(num_correct)/num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * accuracy))

In [46]:
#training loop
def train(model, optimizer, epochs=1):
    model = model.to(device=device)  # move the model parameters to GPU if in use
    for e in range(epochs):
        print("Epoch %d" % (e))
        for t, (x, y) in enumerate(loader_train):
            model.train() #set model to training mode
            x = x.to(device=device, dtype=dtype)  # move to GPU if in use
            y = y.to(device=device, dtype=torch.long)

            #get model predictions and compute loss (cross-entropy)
            scores = model(x)
            loss = F.cross_entropy(scores,y)

            #zero all gradients that optimizer updates (running gradients will be applied if we do not do this)
            optimizer.zero_grad()

            #backward pass
            loss.backward()

            #update model parameters
            optimizer.step()

            # print results every print_every iterations
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                checkAccuracy(loader_val, model)
                print()
    

In [47]:
#test model
learning_rate = 3e-4
in_channel = 3
channel_1 = 32
channel_2 = 64
num_classes = 10

model = BasicNet(in_channel,channel_1,channel_2,num_classes)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model,optimizer)


Epoch 0
Iteration 0, loss = 2.2867
Checking accuracy on validation set
Got 80 / 1000 correct (8.00%)

Iteration 100, loss = 1.6309
Checking accuracy on validation set
Got 436 / 1000 correct (43.60%)

Iteration 200, loss = 1.3448
Checking accuracy on validation set
Got 484 / 1000 correct (48.40%)

Iteration 300, loss = 1.4642
Checking accuracy on validation set
Got 515 / 1000 correct (51.50%)

Iteration 400, loss = 1.2168
Checking accuracy on validation set
Got 555 / 1000 correct (55.50%)

Iteration 500, loss = 1.1370
Checking accuracy on validation set
Got 565 / 1000 correct (56.50%)

Iteration 600, loss = 1.0400
Checking accuracy on validation set
Got 586 / 1000 correct (58.60%)

Iteration 700, loss = 1.1179
Checking accuracy on validation set
Got 606 / 1000 correct (60.60%)

